In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn 

from model.SpeechLP import SLP

from utils.Config import Config
from utils.MLS import MLSDataset
from utils.Trainer import Trainer

from torch.utils.data import DataLoader, random_split

In [3]:
Config.display()

Audio Settings:
  Sample Rate: 24000 Hz
  Min Audio Duration: 10 seconds
  Max Audio Duration: 20 seconds

Text Settings:
  Max Token : 512 tokens

Model Settings:
  Model Name: DiTTO-TTS
  Embedding Dim: 1472
  Num Layers: 1
  Attention Heads: 1

Training Settings:
  Batch Size: 8
  Learning Rate: 0.0001
  Betas: [0.9, 0.999]
  Epochs: 10
  Nb samples: 10000
  Device: cuda

Data Settings:
  Train ratio: 0.8
  Val ratio: 0.2
  Train path: C:/Cours-Sorbonne/M2/UE_DEEP/AMAL/Projet/data/mls_french_opus/mls_french_opus/train
  Test path: C:/Cours-Sorbonne/M2/UE_DEEP/AMAL/Projet/data/mls_french_opus/mls_french_opus/test


In [4]:
dataset = MLSDataset(
    data_dir=Config.TRAIN_PATH,
    max_text_token_length=Config.MAX_TOKEN_LENGTH,
    sampling_rate=Config.SAMPLE_RATE,
)

train_set, val_set = random_split(dataset, [Config.TRAIN_RATIO, Config.VAL_RATIO])

train_loader = DataLoader(train_set, batch_size=Config.BATCH_SIZE, shuffle=True, collate_fn=MLSDataset.collate_fn)
val_loader = DataLoader(val_set, batch_size=Config.BATCH_SIZE, shuffle=True, collate_fn=MLSDataset.collate_fn)

In [5]:
model = SLP(Config.MAX_AUDIO_DURATION, Config.NHEAD ,Config.NUM_LAYERS).to(Config.DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW

trainer = Trainer()
trainer.set_model(model, name="SLP")\
    .set_criterion(criterion)\
    .set_optimizer(optimizer)\
    .fit(
        train_data=train_loader, validation_data=val_loader, 
        epochs=Config.EPOCHS, learning_rate=Config.LEARNING_RATE, checkpoint_interval=1        
    )

c:\Users\halim\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\encodec\modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


[INFO] Model's device is : cuda


  0%|          | 0/10 [00:00<?, ?it/s]


ValueError: not enough values to unpack (expected 3, got 2)